# Instructions:

First run all of the cells that import the required modules and define all required functions. Then in the final cell, replace the "smile" input of the getPredictions function with the SMILES string of the molecule that you want to predict the column density of. Also replace the isIso input with True if the molecule is a rare isotopologue. Please reach out if something is not working properly (zfried@mit.edu). 

In [ ]:
#installing Mol2vec

!pip install git+https://github.com/samoturk/mol2vec

In [ ]:
# Import sklearn modules necessary for following sections

from pathlib import Path
from tempfile import NamedTemporaryFile
import fileinput
import os
import rdkit
import pandas as pd
import numpy as np
import mol2vec
from mol2vec import features
from mol2vec import helpers
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import pkg_resources
import gensim
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import seaborn as sns
import matplotlib.pyplot as plt
import gensim.downloader
from rdkit import RDLogger   
RDLogger.DisableLog('rdApp.*') # turn off RDKit warning message 
from rdkit.Chem import Descriptors
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score, LeaveOneOut, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, ConstantKernel as C, RationalQuadratic, Matern
import numpy as np
import itertools
import pickle

In [ ]:
def sentences2vec(sentences, model, unseen=None):
    
    """Generate vectors for each sentence (list) in a list of sentences. Vector is simply a
    sum of vectors for individual words.
   
    Parameters
    ----------
    sentences : list, array
        List with sentences
    model : word2vec.Word2Vec
        Gensim word2vec model
    unseen : None, str
        Keyword for unseen words. If None, those words are skipped.
        https://stats.stackexchange.com/questions/163005/how-to-set-the-dictionary-for-text-analysis-using-neural-networks/163032#163032
    Returns
    -------
    np.array
    """
    #keys = list(model.wv.vocab.key)
    keys = set(model.wv.index_to_key)
    vec = []
    if unseen:
        unseen_vec = model.wv.get_vector(unseen)

    for sentence in sentences:
        if unseen:
            vec.append(sum([model.wv.get_vector(y) if y in set(sentence) & keys
                       else unseen_vec for y in sentence]))
        else:
            vec.append(sum([model.wv.get_vector(y) for y in sentence
                            if y in set(sentence) & keys]))
    return np.array(vec)

In [ ]:
'''
Each of the functions defined in this cell are helper functions
for the addIsotopologueData() function below. They each use the RDKit
module to return information about the chemical environment of the isotopic
substitution
'''

def get2BondsOver(smile):
    mol = Chem.MolFromSmiles(smile)
    mol = Chem.AddHs(mol)

    #bonds = Chem.GetBonds(mol)
    # Iterate over the atoms
    dList = []
    neighborList = []
    for atom in mol.GetAtoms():
        if atom.GetIsotope() == 2:
            indivD = []
            neighbors = atom.GetNeighbors()
            for i in neighbors:
                indivD.append(i.GetIdx())
                if i.GetIdx() not in neighborList:
                    neighborList.append(i.GetIdx())
            dList.append(indivD)

        # For each atom, set the property "atomNote" to a index+1 of the atom
        atom.SetProp("atomNote", str(atom.GetIdx()))

    oCount = 0
    cCount = 0
    nCount = 0

    neighborList2 = []    
    for z in neighborList:
        atom = mol.GetAtoms()[z]
        neighbors = atom.GetNeighbors()
        for neighbor in neighbors:
            symbol = neighbor.GetSymbol()
            if symbol == "O":
                oCount +=1
            if symbol == "C":
                cCount += 1
            if symbol == "N":
                nCount += 1

    Draw.MolsToGridImage([mol])
    
    return oCount, cCount, nCount


def getCDO(smile, m):
    
    cdo = 0
    
    neighborList = []
    for atom in m.GetAtoms():
        if atom.GetSymbol() == "H" and atom.GetIsotope() == 2:
            for i in atom.GetNeighbors():
                neighborList.append(i)
    neighborIdx = []            
    for nei in neighborList:
        neighborIdx.append(nei.GetIdx())
        
    for i in range(len(neighborList)):
        nei = neighborList[i]
        idx = neighborIdx[i]
        
        oIdx = []
        for nei2 in nei.GetNeighbors():
            if nei2.GetSymbol() == "O":
                oIdx.append(nei2.GetIdx())
                
        for bond in m.GetBonds():
            beginIdx = bond.GetBeginAtomIdx()
            endIdx = bond.GetEndAtomIdx()
            idxTuple = (beginIdx, endIdx)
            
            for o in oIdx:
                if o in idxTuple and idx in idxTuple:
                    bondType = bond.GetBondTypeAsDouble()
                    if bondType == 2:
                        cdo += 1
                        
    return cdo


def get13CO(smile, m):
    count = 0
    neighborList = []
    for atom in m.GetAtoms():
        if atom.GetSymbol() == "C" and atom.GetIsotope() == 13:
            for i in atom.GetNeighbors():
                neighborList.append(i)
                
    for nei in neighborList:
        if nei.GetSymbol() == "O":
            count += 1
    
    return count

def getCD(smile, m):
    count = 0
    neighborList = []
    for atom in m.GetAtoms():
        if atom.GetSymbol() == "H" and atom.GetIsotope() == 2:
            for i in atom.GetNeighbors():
                neighborList.append(i)
                
    for nei in neighborList:
        if nei.GetSymbol() == "C":
            count += 1
    
    return count


def getOD(smile, m):
    count = 0
    neighborList = []
    for atom in m.GetAtoms():
        if atom.GetSymbol() == "H" and atom.GetIsotope() == 2:
            for i in atom.GetNeighbors():
                neighborList.append(i)
                
    for nei in neighborList:
        if nei.GetSymbol() == "O":
            count += 1
    
    return count



In [ ]:
def addIsotopologueData(clusterSmiles, vectors):
    '''
    This function takes a SMILES string and corresponding feature vector
    as its input. It then adds 19 dimensions to the feature vector that encode
    isotopic substitution.
    
    The function returns a list containing the 89 dimensional feature vector
    
    '''
    
    
    d = []
    s34 = []
    s33 = [] 
    s36 = []
    c13 = []
    o17 = []
    o18 = []
    n15 = []
    cl37 = [] 
    sp = []
    sp2 = []
    sp3 = []
    c13oList = []
    cdList = []
    odList = []
    heavyAtoms = []
    oAdj = []
    cAdj = []
    cdo = []
    
    
    for smi in clusterSmiles:
        #i = clusterSmiles.index(smi)
        sp2Count = 0
        sp3Count = 0
        spCount = 0

        d.append(smi.count('2H'))
        s34.append(smi.count('34S'))
        s33.append(smi.count('33S'))
        s36.append(smi.count('36S'))
        c13.append(smi.count('13C'))
        o17.append(smi.count('17O'))
        o18.append(smi.count('18O'))
        n15.append(smi.count('15N'))
        cl37.append(smi.count('37Cl'))

        m = Chem.MolFromSmiles(smi)
        
        if smi.count('13C') != 0:
            for atom in m.GetAtoms():
                if atom.GetSymbol() == "C" and atom.GetIsotope() == 13:
                    hybrid = atom.GetHybridization()
                    hybrid = str(hybrid)
                    if hybrid == "SP2":
                        sp2Count += 1
                    if hybrid == "SP3":
                        sp3Count += 1
                    if hybrid == "SP":
                        spCount += 1

        sp.append(spCount)
        sp2.append(sp2Count)
        sp3.append(sp3Count)
        
        
        c13o = get13CO(smi, m)
        c13oList.append(c13o)
        
        cd = getCD(smi, m)
        cdList.append(cd)
        
        od = getOD(smi,m)
        odList.append(od)
        

        totalAtoms = m.GetNumHeavyAtoms()
        heavyAtoms.append(totalAtoms)


        o, c, n = get2BondsOver(smi)
        oAdj.append(o)
        cAdj.append(c)
        
        
        cdo.append(getCDO(smi, m))
        
        
    for i in range(len(vectors)):
        vectors[i].append(d[i])
        vectors[i].append(s34[i])
        vectors[i].append(s33[i])
        vectors[i].append(s36[i])
        vectors[i].append(c13[i])
        vectors[i].append(o17[i])
        vectors[i].append(o18[i])
        vectors[i].append(n15[i])
        vectors[i].append(cl37[i])
        vectors[i].append(sp[i])
        vectors[i].append(sp2[i])
        vectors[i].append(sp3[i])
        vectors[i].append(c13oList[i])
        vectors[i].append(cdList[i])
        vectors[i].append(odList[i])
        vectors[i].append(heavyAtoms[i])
        vectors[i].append(oAdj[i])
        vectors[i].append(cAdj[i])
        vectors[i].append(cdo[i])
        
        
    return vectors

In [ ]:
def getFeatureVector(smile):
    
    '''
    This function takes a SMILES string of a main isotopologue as an input. 
    It then uses a trained Mol2vec model to create a 70
    dimensional feature vector for the inputted molecule. 
    It then calls the addIsotopologueData() function to add 19 dimensions
    that encode isotopic substitution.
    
    The function returns a list containing the resulting 89 dimensional feature vector
    
    '''
    
    modelPath = os.path.join(os.getcwd(), 'mol2vec_model_final_70.pkl')
    model = word2vec.Word2Vec.load(modelPath)

    detectSmiles = [smile]

    molNames = [Chem.MolFromSmiles(smile) for smile in detectSmiles]
    sentences = [features.mol2alt_sentence(mole,1) for mole in molNames]
    vectors = sentences2vec(sentences,model)
    vectors = [list(i) for i in vectors]
    fullVectors = addIsotopologueData(detectSmiles, vectors)
    
    return fullVectors

In [ ]:
def getFeatureVectorIso(smile):
    
    '''
    This function takes a SMILES string of a rare istopologue as an input. 
    It then uses a trained Mol2vec model to create a 70
    dimensional feature vector for the inputted molecule. 
    It then calls the addIsotopologueData() function to add 19 dimensions
    that encode isotopic substitution.
    
    The function returns a list containing the resulting 89 dimensional feature vector
    
    '''
    
    isoSmiles = [smile]
    
    mol = Chem.MolFromSmiles(smile)
    for atom in mol.GetAtoms():
        atom.SetIsotope(0)
        
    mainSmile = Chem.MolToSmiles(mol)
    
    
    
    
    
    modelPath = os.path.join(os.getcwd(), 'mol2vec_model_final_70.pkl')
    model = word2vec.Word2Vec.load(modelPath)

    detectSmiles = [mainSmile]

    molNames = [Chem.MolFromSmiles(smile) for smile in detectSmiles]
    sentences = [features.mol2alt_sentence(mole,1) for mole in molNames]
    vectors = sentences2vec(sentences,model)
    vectors = [list(i) for i in vectors]
    fullVectors = addIsotopologueData(isoSmiles, vectors)
    
    return fullVectors

In [ ]:
import pickle

def getPrediction(smile, isIso, model = "GPR"):
    
    '''
    This function takes 3 inputs:
    
    1) SMILES string of the molecule to predict
    
    2) Whether the molecule is a rare isotopologue or not. Setting iso = True
    denotes a rare isotopologue. Setting iso = False denotes a main isotopologue
    
    3) A string that defines the model to be used for the prediction
    
    The options for the model input are:
    
    "GPR": This is the default and uses a trained Gaussian process regression model,
    "BR": This uses a trained Bayesian ridge regression model,
    "GBR": This uses a trained gradient boosting regression model
    
    ___
    
    The getFeatureVector() or getFeatureVectorIso() 
    function is then called to generate an 89 dimensional 
    feature vector for the inputted molecule that includes isotopic information
    
    The column density is then predicted.
    
    If a GPR or BR model is used, a 3-tuple is returned that is formatted as follows:
    
    (SMILES string, log10 predcited column density, 1 sigma prediction uncertainty)
    
    If a GBR model is used, a tuple is returned that is formatted as follows:
    
    (SMILES string, log10 predcited column density)
    
    '''
    
    smileList = [smile]
    
    if isIso == False:
        X_test = getFeatureVector(smile)
    else:
        X_test = getFeatureVectorIso(smile)
    
    scalerPath = os.path.join(os.getcwd(), 'scaler.pkl')
    sc = pickle.load(open(scalerPath,'rb'))
    X_test_scaled = sc.transform(X_test)
    
    if model == "GPR":
        modelPath = os.path.join(os.getcwd(), 'gpr_model.pkl')
    if model == "BR":
        modelPath = os.path.join(os.getcwd(), 'br_model.pkl') 
    if model == "GBR":
        modelPath = os.path.join(os.getcwd(), 'gbr_model.pkl') 
    
    
    result = pickle.load(open(modelPath, 'rb'))
    
    if model == "GPR" or model == "BR":
        validationResults, sd = result.predict(X_test_scaled, return_std = True)
        completeList = [(smileList[i], validationResults[i], sd[i]) for i in range(len(X_test))]
        
    
    if model == "GBR":
        validationResults = result.predict(X_test_scaled)
        completeList = [(smileList[i], validationResults[i]) for i in range(len(X_test))]
    
    return completeList[0]

In [ ]:
'''
Run this cell to get a column density prediction.
You need to replace the "smile" input with the SMILES string of the 
molecule you are interested in. 

'''

prediction = getPrediction(smile = "CHANGE THIS", isIso = False, model = "GPR")
if len(prediction) == 3:
    print("SMILES: " + prediction[0])
    print("Predicted Log Column Density: " + str(prediction[1]))
    print("Prediction Uncertainty: " + str(prediction[2]))
if len(prediction) == 2:
    print("SMILES: " + prediction[0])
    print("Predicted Log Column Density: " + str(prediction[1]))
